In [1]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import warnings
import json
import os

warnings.filterwarnings('ignore')

# Set up environment variables
%env ASTRA_DB_SECURE_BUNDLE_PATH=C:\ineuron projects\secure-connect-flightfareprediction.zip
%env ASTRA_DB_APPLICATION_TOKEN=C:\ineuron projects\flightfareprediction-token.json

# Cassandra connection setup

# Get the environment variables
secure_bundle_path = os.getenv('ASTRA_DB_SECURE_BUNDLE_PATH')
application_token_path = os.getenv('ASTRA_DB_APPLICATION_TOKEN')

# Read the application token from the JSON file
with open(application_token_path, 'r') as token_file:
    application_token = json.load(token_file)['token']

# Set up the connection configuration
cloud_config = {
    'secure_connect_bundle': secure_bundle_path
}
auth_provider = PlainTextAuthProvider('token', application_token)

# Create the cluster and session
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Test the connection by listing the keyspaces
keyspaces = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")
print("Available keyspaces:")
for keyspace in keyspaces:
    print(keyspace.keyspace_name)

# Set the keyspace to 'flightprice'
session.set_keyspace('flightprice')

# Create the trainingdata table
session.execute("""
CREATE TABLE IF NOT EXISTS trainingdata (
    id UUID PRIMARY KEY,
    date_of_journey TEXT,
    additional_info TEXT,
    airline TEXT,
    arrival_time TEXT,
    dep_time TEXT,
    destination TEXT,
    duration TEXT,
    price FLOAT,
    route TEXT,
    source TEXT,
    total_stops TEXT
)
""")
print("Table 'trainingdata' created.")

# Insert sample data into the trainingdata table for testing
session.execute("""
INSERT INTO trainingdata (id, date_of_journey, additional_info, airline, arrival_time, dep_time, destination, duration, price, route, source, total_stops)
VALUES (uuid(), '2023-05-01', 'No info', 'Air India', '23:30', '21:30', 'Cochin', '2h', 5000.0, 'DEL -> BOM -> COK', 'Delhi', '1 stop')
""")

# Query data from the trainingdata table
query = "SELECT * FROM trainingdata;"
result_set = session.execute(query)

# Convert result_set to a DataFrame
rows = []
for row in result_set:
    rows.append(row._asdict())

df = pd.DataFrame(rows)

# Display the DataFrame
df.head()


env: ASTRA_DB_SECURE_BUNDLE_PATH=C:\ineuron projects\secure-connect-flightfareprediction.zip
env: ASTRA_DB_APPLICATION_TOKEN=C:\ineuron projects\flightfareprediction-token.json
Available keyspaces:
flightprice
data_endpoint_auth
system_auth
system_schema
datastax_sla
system
system_traces
Table 'trainingdata' created.


,id,additional_info,airline,arrival_time,date_of_journey,dep_time,destination,duration,price,route,source,total_stops
0,1e49d1c7-8d5e-4bce-ab05-37c64323d5fa,No info,Air India,23:30,2023-05-01,21:30,Cochin,2h,5000.0,DEL -> BOM -> COK,Delhi,1 stop
1,10c8bcd0-e64f-4773-8f39-7d4c6965f12b,No info,Air India,23:30,2023-05-01,21:30,Cochin,2h,5000.0,DEL -> BOM -> COK,Delhi,1 stop


In [21]:
# Don't forget to close the connection when done
session.shutdown()
cluster.shutdown()